---
title: "EDA"
---

In [ ]:
import pandas as pd
import ast
import networkx as nx
from itertools import combinations
from collections import defaultdict


In [ ]:
# Load your CSV
df = pd.read_csv("../data/raw/movies.csv")


# Parse actors and directors into lists
df["Director"] = df["Director"].apply(lambda x: [d.strip() for d in x.split(",")] if isinstance(x, str) else [])



In [ ]:
collaborator_to_movies = defaultdict(set)

for idx, row in df.iterrows():
    movie_id = row["IMDB_ID"]
    
    for actor in row["Actors"]:
        collaborator_to_movies[actor.strip()].add(movie_id)

    for director in row["Director"]:
        collaborator_to_movies[director.strip()].add(movie_id)

In [ ]:
G = nx.Graph()

# Add movie nodes with metadata
for idx, row in df.iterrows():
    G.add_node(row["IMDB_ID"], title = row["Title"], year = row["Year"])

# Create edges between movies that share collaborators
for collaborator, movies in collaborator_to_movies.items():
    movie_pairs = combinations(movies, 2)
    for movie_1, movie_2 in movie_pairs:
        if G.has_edge(movie_1, movie_2):
            # Add to existing set of collaborators
            G[movie_1][movie_2]["collaborators"].add(collaborator)
        else:
            G.add_edge(movie_1, movie_2, collaborators = {collaborator})
